In [1]:
!pip install google-generativeai datasets pandas tqdm

In [2]:
from datasets import load_dataset

# Load the GretelAI synthetic text-to-sql dataset
dataset = load_dataset("gretelai/synthetic_text_to_sql", split="train")

print("Total samples:", len(dataset))
print(dataset[0])

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Total samples: 100000
{'id': 5097, 'domain': 'forestry', 'domain_description': 'Comprehensive data on sustainable forest management, timber production, wildlife habitat, and carbon sequestration in forestry.', 'sql_complexity': 'single join', 'sql_complexity_description': 'only one join (specify inner, outer, cross)', 'sql_task_type': 'analytics and reporting', 'sql_task_type_description': 'generating reports, dashboards, and analytical insights', 'sql_prompt': 'What is the total volume of timber sold by each salesperson, sorted by salesperson?', 'sql_context': "CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-

In [3]:
pip install -q -U google-genai

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.13.5 requires anyio<4,>=3.1.0, but you have anyio 4.9.0 which is incompatible.
jupyter-server 1.13.5 requires pywinpty<2; os_name == "nt", but you have pywinpty 2.0.10 which is incompatible.


In [5]:
print(len(dataset))

100000


In [2]:
import google.generativeai as genai
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm
import time
import os

# Configure Gemini
genai.configure(api_key="Add the api key")  
model = genai.GenerativeModel("gemini-1.5-flash")

# Load dataset from HuggingFace
dataset = load_dataset("gretelai/synthetic_text_to_sql", split="train")
print(f"Total rows: {len(dataset)}")

# Limit number of samples
subset = dataset.select(range(100000))  

# Resume from last saved progress
partial_path = "gemini_sql_results_partial.csv"
if os.path.exists(partial_path):
    df_existing = pd.read_csv(partial_path)
    start_idx = len(df_existing)
    results = df_existing.to_dict("records")
    print(f"Resuming from index {start_idx}")
else:
    start_idx = 0
    results = []
    print("Starting from scratch")

# Loop over remaining samples
for idx in tqdm(range(start_idx, len(subset))):
    sample = subset[idx]
    context = sample["sql_context"]
    question = sample["sql_prompt"]
    gold_sql = sample["sql"]

    # Gemini prompt
    prompt = f"""
You are a Text-to-SQL generator.
Given the following database schema and sample data:

{context}

Write an SQL query to answer this question:
"{question}"

Respond with ONLY the SQL query. No extra explanation.
SQL:
"""

    try:
        response = model.generate_content(prompt)
        generated_sql = response.text.strip()
    except Exception as e:
        if "429" in str(e):
            print("Rate limit hit. Sleeping for 30 seconds...")
            time.sleep(30)
            try:
                response = model.generate_content(prompt)
                generated_sql = response.text.strip()
            except Exception as e2:
                generated_sql = f"ERROR: {e2}"
        else:
            generated_sql = f"ERROR: {e}"

    # Clean markdown
    if generated_sql.startswith("```sql"):
        generated_sql = generated_sql.replace("```sql", "").replace("```", "").strip()

    results.append({
        "sql_prompt": question,
        "sql_context": context,
        "gold_sql": gold_sql,
        "generated_sql": generated_sql,
        "exact_match": gold_sql.strip().lower() == generated_sql.strip().lower()
    })

    # Save every 100
    if (idx + 1) % 100 == 0:
        pd.DataFrame(results).to_csv(partial_path, index=False)
        print(f"Saved {idx + 1} results to {partial_path}")

    # Rest every 1000
    if (idx + 1) % 500 == 0:
        print("Sleeping for 3 minutes after 1000 entries...")
        time.sleep(180)

    # Avoid quota overflow
    time.sleep(2)

# Final Save
df_final = pd.DataFrame(results)
df_final.to_csv("gemini_sql_results.csv", index=False)
print("Final results saved to gemini_sql_results.csv")

# Accuracy Report
accuracy = df_final["exact_match"].mean()
print(f"Exact Match Accuracy: {accuracy:.2%}")


Total rows: 100000
Resuming from index 2300


  0%|                                                                            | 16/97700 [00:43<72:16:05,  2.66s/it]

Rate limit hit. Sleeping for 30 seconds...


  0%|                                                                            | 32/97700 [01:56<72:02:15,  2.66s/it]

Rate limit hit. Sleeping for 30 seconds...


...

Rate limit hit. Sleeping for 30 seconds...


  2%|█                                                                      | 1525/97700 [2:01:54<861:13:33, 32.24s/it]

Rate limit hit. Sleeping for 30 seconds...


  2%|█                                                                      | 1525/97700 [2:02:24<128:39:58,  4.82s/it]


KeyboardInterrupt: 

In [3]:
import pandas as pd

# Load the CSV
df = pd.read_csv('gemini_sql_results_partial.csv')

# Compute Exact Match Accuracy
accuracy = df["exact_match"].mean()

# Show accuracy
print(f"Exact Match Accuracy: {accuracy:.2%}")


🎯 Exact Match Accuracy: 12.39%
